# Create and run a Machine Learning model as a custom-script

This notebook showcases how to create a Machine Learning (ML) custom-script for water detection. The workflow uses [eo-learn](https://eo-learn.readthedocs.io/en/latest/) to process the data and [LightGBM](https://lightgbm.readthedocs.io/en/latest/) to train a ML model for water classification given Seninel-2 band and index values. The resulting custom-script can be used in [the Sentinel Hub EOBrowser](https://www-test.sentinel-hub.com/explore/eobrowser/), in the [multi-temporal instance of Sentinel Playground](https://apps.sentinel-hub.com/sentinel-playground-temporal/?source=S2&lat=40.4&lng=-3.730000000000018&zoom=12&preset=1-NATURAL-COLOR&layers=B04,B03,B02&maxcc=20&gain=1.0&temporal=true&gamma=1.0&time=2015-01-01%7C2019-10-02&atmFilter=&showDates=false) and as evalscript in the [Sentinel Hub process API](https://docs.sentinel-hub.com/api/latest/api/process/).

The workflow is as follows:

 * download training and testing data
 * prepare samples for ML algorithm
 * train ML model
 * export trained model as custom-script

In [1]:
# Jupyter notebook related
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Built-in modules
import urllib.request
import shutil
import zipfile

# # Basics of Python data handling and visualization
import numpy as np
import geopandas as gpd
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec
# from matplotlib.colors import ListedColormap, BoundaryNorm
# from mpl_toolkits.axes_grid1 import make_axes_locatable
# from shapely.geometry import Polygon
from tqdm.auto import tqdm

# Machine learning 
from sklearn.cluster import DBSCAN
import lightgbm as lgbm
# import joblib
from sklearn import metrics
# from sklearn import preprocessing

# # Imports from eo-learn and sentinelhub-py
from eolearn.core import EOPatch, EOTask, LinearWorkflow, FeatureType, OverwritePermission, \
    LoadTask, SaveTask, EOExecutor, ExtractBandsTask, MergeFeatureTask
from eolearn.features import NormalizedDifferenceIndexTask
from eolearn.geometry import PointSamplingTask #, ErosionTask, VectorToRaster
# from eolearn.io import SentinelHubInputTask, ExportToTiff
# from eolearn.mask import AddMultiCloudMaskTask, AddValidDataMaskTask
# from sentinelhub import UtmZoneSplitter, BBox, CRS, DataSource

## Download the data

* used to improve the blue-bot observatory
* collected with the Classification App
* available on bucket
* info on what they contain

Set up url and data paths

In [2]:
DATA_URL = 'http://queryplanet.sentinel-hub.com/water-labels'

DATA_INFO = f'{DATA_URL}/data-info.geojson'
EOP_URL = f'{DATA_URL}/eopatches.zip'

EOP_ZIP = './eopatches.zip'
EOP_DIR = '.'

In [3]:
S2_BANDS = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11', 'B12']
S2_BANDS_STR = ','.join(S2_BANDS)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

TRAIN_RATIO = .8

N_WORKERS = 4

N_SAMPLES = 100
REF_LABELS = [0, 1]  

In [4]:
gdf = gpd.read_file(DATA_INFO)

In [5]:
gdf.head()

,has_DEM,has_S1_ASC,has_S1_DES,has_S2,task_id,timestamp,window_height,window_width,geometry
0,1,0,0,1,6b73fd74a2eb11e994fbf0db728b8d14,2016-09-26,64,64,"POLYGON ((65.84065 52.68916, 65.84065 52.69491..."
1,1,0,1,1,0a27f4aea2eb11e9bfdaa9140581204c,2017-08-12,64,64,"POLYGON ((64.47018 50.95125, 64.47018 50.95701..."
2,1,1,1,1,9b60193ea24111e98d7d929084d604de,2019-03-23,64,64,"POLYGON ((31.15408 38.61106, 31.15408 38.61600..."
3,1,1,1,1,9afaabaea24111e983d185262381d01a,2018-05-31,64,64,"POLYGON ((-1.60285 38.21084, -1.60285 38.21558..."
4,1,0,1,1,9dfb5254a24111e9a087737fe71fcff1,2018-05-27,64,64,"POLYGON ((48.22342 48.99523, 48.22342 49.00099..."


In [6]:
len(gdf), len(gdf[gdf.has_S2==1])

(7671, 7671)

### Download and unzip file with eopatches

In [7]:
print(f'Downloading {EOP_URL} to {EOP_ZIP} ..')
with urllib.request.urlopen(EOP_URL) as response, open(EOP_ZIP, 'wb') as zip_file:
    shutil.copyfileobj(response, zip_file)
    
print(f'Unzipping {EOP_ZIP} to {EOP_DIR} ..')
with zipfile.ZipFile(EOP_ZIP, 'r') as zip_file:
    zip_file.extractall(EOP_DIR)

Unzipping ./eopatches.zip to . ..


In [8]:
ls -lth {EOP_DIR}/eopatches | wc -l 

7672


## Set-up and run feature processing workflow with `eo-learn`

* load eopatches
* keep only B02, B03 and B04
* add NDWI and NDMI
* sample pixels
* save sampled arrays only

Build name of eopatches, given by index of `data_info`

In [9]:
eopatch_names = [f'eopatch-{idx:04d}' for idx in np.arange(len(gdf))]

Load an eopathc and inspect content

In [10]:
eop = EOPatch.load(f'{EOP_DIR}/eopatches/{eopatch_names[1]}')
eop

EOPatch(
  data: {
    BANDS-S2-L1C: numpy.ndarray(shape=(1, 64, 64, 13), dtype=float32)
    FEATURES_SAMPLED: numpy.ndarray(shape=(1, 100, 1, 5), dtype=float32)
    TRUE-COLOR-S1-IW-DES: numpy.ndarray(shape=(1, 64, 64, 3), dtype=float32)
  }
  mask: {
    IS_DATA: numpy.ndarray(shape=(1, 64, 64, 1), dtype=bool)
    IS_DATA_S1_IW_DES: numpy.ndarray(shape=(1, 64, 64, 1), dtype=bool)
    IS_DATA_SAMPLED: numpy.ndarray(shape=(1, 100, 1, 1), dtype=bool)
  }
  scalar: {}
  label: {}
  vector: {}
  data_timeless: {
    DEM: numpy.ndarray(shape=(64, 64, 1), dtype=float32)
  }
  mask_timeless: {
    water_label: numpy.ndarray(shape=(64, 64, 1), dtype=uint8)
    water_label_SAMPLED: numpy.ndarray(shape=(100, 1, 1), dtype=uint8)
  }
  scalar_timeless: {}
  label_timeless: {}
  vector_timeless: {}
  meta_info: {
    TRUE-COLOR-S1-IW-DES: datetime.timedelta(-2, 6658)
    maxcc: 1.0
    service_type: 'wms'
    size_x: 64
    size_y: 64
    time_difference: datetime.timedelta(1)
    time_interval: (

In [11]:
BANDS_FEATURE = FeatureType.DATA, 'BANDS-S2-L1C'
IS_DATA_FEATURE = FeatureType.MASK, 'IS_DATA'

BANDS_SUB_FEATURE = FeatureType.DATA, 'BANDS-SUBSET'
NDWI_FEATURE = FeatureType.DATA, 'NDWI'
NDMI_FEATURE = FeatureType.DATA, 'NDMI'

FEATURES = FeatureType.DATA, 'FEATURES'

LABELS = FeatureType.MASK_TIMELESS, 'water_label'

FEATURES_SAMPLED = FeatureType.DATA, 'FEATURES_SAMPLED'
IS_DATA_SAMPLED = FeatureType.MASK, 'IS_DATA_SAMPLED'
LABELS_SAMPLED = FeatureType.MASK_TIMELESS, 'water_label_SAMPLED'

In [12]:
# Load eopatch
load_task = LoadTask(f'{EOP_DIR}/eopatches/')

# Keep only B02, B03, B04
extract_task = ExtractBandsTask(input_feature=BANDS_FEATURE, 
                                output_feature=BANDS_SUB_FEATURE, 
                                bands=[S2_BANDS.index(s2b) for s2b in ['B02', 'B03', 'B04']])

# NDWI = (B03 - B08) / (B03 + B08)
ndwi_task = NormalizedDifferenceIndexTask(input_feature=BANDS_FEATURE,
                                          output_feature=NDWI_FEATURE,
                                          bands=[S2_BANDS.index(s2b) for s2b in ['B03', 'B08']])

# NDMI = (B08 - B11) / (B08 + B11)
ndmi_task = NormalizedDifferenceIndexTask(input_feature=BANDS_FEATURE,
                                          output_feature=NDMI_FEATURE,
                                          bands=[S2_BANDS.index(s2b) for s2b in ['B08', 'B11']])

# Merge features as [B02, B03, B04, NDWI, NDMI]
merge_task = MergeFeatureTask(input_features=[BANDS_SUB_FEATURE, NDWI_FEATURE, NDMI_FEATURE],
                              output_feature=FEATURES)

# Task for pixels' sampling
sampling_task = PointSamplingTask(
    n_samples=N_SAMPLES, 
    ref_mask_feature=LABELS[1], 
    ref_labels=REF_LABELS, 
    sample_features=[  # tag fields to sample
        FEATURES,
        IS_DATA_FEATURE,
        LABELS
    ])

# Add to exising EOPatch only the sampled features and labels
save_task = SaveTask(
    f'{EOP_DIR}/eopatches/', 
    features=[FEATURES_SAMPLED, IS_DATA_SAMPLED, LABELS_SAMPLED],
    overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)

In [13]:
# Define the workflow
workflow = LinearWorkflow(
    load_task,
    extract_task,
    ndwi_task,
    ndmi_task,
    merge_task,
    sampling_task,
    save_task
)

In [14]:
%%time

# Create list of arguments to be passed at run-time
execution_args = []
for idx, eop_name in enumerate(eopatch_names):
    execution_args.append({
        load_task: {'eopatch_folder': eop_name},
        sampling_task: {'seed': idx},
        save_task: {'eopatch_folder': eop_name}
    })
    
# Execute workflow in parallel, e.g. each EOPatch is process in parallel in dedicated processes
executor = EOExecutor(workflow, execution_args, save_logs=True)
executor.run(workers=N_WORKERS, multiprocess=True)

# Make report to check possible failures
executor.make_report()


CPU times: user 1min 48s, sys: 2.78 s, total: 1min 51s
Wall time: 3min 26s


In [15]:
eop = EOPatch.load(f'{EOP_DIR}/eopatches/{eopatch_names[1]}')
eop

EOPatch(
  data: {
    BANDS-S2-L1C: numpy.ndarray(shape=(1, 64, 64, 13), dtype=float32)
    FEATURES_SAMPLED: numpy.ndarray(shape=(1, 100, 1, 5), dtype=float32)
    TRUE-COLOR-S1-IW-DES: numpy.ndarray(shape=(1, 64, 64, 3), dtype=float32)
  }
  mask: {
    IS_DATA: numpy.ndarray(shape=(1, 64, 64, 1), dtype=bool)
    IS_DATA_S1_IW_DES: numpy.ndarray(shape=(1, 64, 64, 1), dtype=bool)
    IS_DATA_SAMPLED: numpy.ndarray(shape=(1, 100, 1, 1), dtype=bool)
  }
  scalar: {}
  label: {}
  vector: {}
  data_timeless: {
    DEM: numpy.ndarray(shape=(64, 64, 1), dtype=float32)
  }
  mask_timeless: {
    water_label: numpy.ndarray(shape=(64, 64, 1), dtype=uint8)
    water_label_SAMPLED: numpy.ndarray(shape=(100, 1, 1), dtype=uint8)
  }
  scalar_timeless: {}
  label_timeless: {}
  vector_timeless: {}
  meta_info: {
    TRUE-COLOR-S1-IW-DES: datetime.timedelta(-2, 6658)
    maxcc: 1.0
    service_type: 'wms'
    size_x: 64
    size_y: 64
    time_difference: datetime.timedelta(1)
    time_interval: (

## Create train/cval/test sets

Split eopatches into train/test sets based on location and timestamps, e.g. images from same and different timestamps will end up in same set.

In [16]:
gdf['x']=gdf.geometry.apply(lambda g: g.centroid.x)
gdf['y']=gdf.geometry.apply(lambda g: g.centroid.y)
db = DBSCAN(eps=0.1, min_samples=1, algorithm='ball_tree', metric='euclidean').fit(gdf[['x', 'y']].to_numpy())
for label in set(db.labels_):
    class_member_mask = (db.labels_ == label)
    gdf.loc[class_member_mask,'cluster_id'] = label
    
gdf['group'] = gdf.groupby(by=['cluster_id','timestamp'],as_index=False).ngroup()
gdf.drop(columns=['x','y','cluster_id'], inplace=True)

train_ids = set(np.where(np.random.rand(gdf.group.nunique())<=TRAIN_RATIO)[0])

gdf['isin_train'] = gdf.group.isin(train_ids)

In [17]:
gdf.head()

,has_DEM,has_S1_ASC,has_S1_DES,has_S2,task_id,timestamp,window_height,window_width,geometry,group,isin_train
0,1,0,0,1,6b73fd74a2eb11e994fbf0db728b8d14,2016-09-26,64,64,"POLYGON ((65.84065 52.68916, 65.84065 52.69491...",1,False
1,1,0,1,1,0a27f4aea2eb11e9bfdaa9140581204c,2017-08-12,64,64,"POLYGON ((64.47018 50.95125, 64.47018 50.95701...",3,True
2,1,1,1,1,9b60193ea24111e98d7d929084d604de,2019-03-23,64,64,"POLYGON ((31.15408 38.61106, 31.15408 38.61600...",4,True
3,1,1,1,1,9afaabaea24111e983d185262381d01a,2018-05-31,64,64,"POLYGON ((-1.60285 38.21084, -1.60285 38.21558...",5,True
4,1,0,1,1,9dfb5254a24111e9a087737fe71fcff1,2018-05-27,64,64,"POLYGON ((48.22342 48.99523, 48.22342 49.00099...",7,False


In [18]:
# load sampled eopatches
eopatches = []

for eopatch_name in tqdm(eopatch_names):
    eopatches.append(EOPatch.load(f'{EOP_DIR}/eopatches/{eopatch_name}', lazy_loading=True))    

eopatches = np.array(eopatches)

In [19]:
# Set the features and the labels for train and test sets
features_train = np.array([eopatch[FEATURES_SAMPLED] for eopatch in eopatches[gdf.isin_train.values]])
labels_train = np.array([eopatch[LABELS_SAMPLED] for eopatch in eopatches[gdf.isin_train.values]])
mask_train = np.array([eopatch[IS_DATA_SAMPLED] for eopatch in eopatches[gdf.isin_train.values]])

features_test = np.array([eopatch[FEATURES_SAMPLED] for eopatch in eopatches[~gdf.isin_train.values]])
labels_test = np.array([eopatch[LABELS_SAMPLED] for eopatch in eopatches[~gdf.isin_train.values]])
mask_test = np.array([eopatch[IS_DATA_SAMPLED] for eopatch in eopatches[~gdf.isin_train.values]])

# get shape
p1, t, w, h, f = features_train.shape
p2, t, w, h, f = features_test.shape
p = p1 + p2

# reshape to n x m
features_train = np.moveaxis(features_train, 1, 3).reshape(p1 * w * h, t * f)
labels_train = np.moveaxis(labels_train, 1, 2).reshape(p1 * w * h, 1).squeeze()
mask_train = np.moveaxis(mask_train, 1, 2).reshape(p1 * w * h, 1).squeeze()

features_test = np.moveaxis(features_test, 1, 3).reshape(p2 * w * h, t * f)
labels_test = np.moveaxis(labels_test, 1, 2).reshape(p2 * w * h, 1).squeeze()
mask_test = np.moveaxis(mask_test, 1, 2).reshape(p2 * w * h, 1).squeeze()

# remove points with no valid data
features_train = features_train[mask_train]
labels_train = labels_train[mask_train]

features_test = features_test[mask_test]
labels_test = labels_test[mask_test]

In [20]:
features_train.shape, labels_train.shape, features_test.shape, labels_test.shape

((610300, 5), (610300,), (156800, 5), (156800,))

In [21]:
np.unique(labels_train, return_counts=True), np.unique(labels_test, return_counts=True)

((array([0, 1], dtype=uint8), array([218908, 391392])),
 (array([0, 1], dtype=uint8), array([ 53657, 103143])))

## Train and evaluate model 

In [22]:
%%time

# Set up the model
model = lgbm.LGBMClassifier(
    objective='binary', 
    n_estimators=10,
    random_state=RANDOM_STATE
)

# train the model
model.fit(features_train, labels_train)

CPU times: user 1.95 s, sys: 23.6 ms, total: 1.97 s
Wall time: 306 ms


In [23]:
# predict the test labels
plabels_test = model.predict(features_test)

In [24]:
class_labels = REF_LABELS
class_names = ['non-water', 'water']
mask = np.in1d(plabels_test, labels_test)
pred = plabels_test[mask]
lbls = labels_test[mask]

f1_scores = metrics.f1_score(lbls, pred, labels=class_labels, average=None)
recall = metrics.recall_score(lbls, pred, labels=class_labels, average=None)
precision = metrics.precision_score(lbls, pred, labels=class_labels, average=None) 

print('Classification accuracy {:.1f}%'.format(100 * metrics.accuracy_score(lbls, pred)))
print('Classification F1-score {:.1f}%'.format(100 * metrics.f1_score(lbls, pred, average='weighted')))
print()
print('             Class              =  F1  | Recall | Precision')
print('         --------------------------------------------------')
for idx, classname in enumerate([class_names[idx] for idx in class_labels]):
    print('         * {0:20s} = {1:2.1f} |  {2:2.1f}  | {3:2.1f}'.format(classname, 
                                                                         f1_scores[idx] * 100, 
                                                                         recall[idx] * 100, 
                                                                         precision[idx] * 100))



Classification accuracy 92.2%
Classification F1-score 92.2%

             Class              =  F1  | Recall | Precision
         --------------------------------------------------
         * non-water            = 88.4 |  86.8  | 90.0
         * water                = 94.1 |  95.0  | 93.3


In [25]:
model.feature_importances_

array([54, 63, 13, 79, 91])

## Convert model to evalscript

## Test evalscript